# Data Cleaning- Google Analytics Customer Prediction

## Table of Contents

    1. Introduction
    2. Importing
    3. Creating new columns from json columns
    4. Cleaning existing columns
    5. Creating new columns
    6. Saving Clean Data

### 1. Introduction

In this notebook, the data from Google Analytics will be formated and cleaned for the purpose of modeling. There are json columns that will be split into multiple columns. Additional two columns will be created, indicating previous customer and if a tranaction was complete. The latter will be the column used for predictions in future notebooks. 

### 2. Importing

In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import math
import time
from datetime import datetime

In [2]:
df = pd.read_csv('train_v2.csv')

/Users/kasey.larsen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### 3. Creating new columns from json columns

In [3]:
#Label json columns
json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [4]:
# Expand json columns
for cols in json_columns:
    df_norm = pd.json_normalize([json.loads(val) for val in df[cols]]).fillna(0)
    df = pd.merge(df, df_norm, left_index=True, right_index=True)

In [5]:
df.shape

(1708337, 64)

In [6]:
# Randomly take a fraction of the data
df = df.sample(frac = 0.05)

In [7]:
pd.set_option('display.max_rows', df.shape[0]+1)
pd.set_option('display.max_columns', None)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85417 entries, 1078605 to 319393
Data columns (total 64 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   channelGrouping                      85417 non-null  object
 1   customDimensions                     85417 non-null  object
 2   date                                 85417 non-null  int64 
 3   device                               85417 non-null  object
 4   fullVisitorId                        85417 non-null  object
 5   geoNetwork                           85417 non-null  object
 6   hits_x                               85417 non-null  object
 7   socialEngagementType                 85417 non-null  object
 8   totals                               85417 non-null  object
 9   trafficSource                        85417 non-null  object
 10  visitId                              85417 non-null  int64 
 11  visitNumber                       

In [9]:
#Drop columns with jsons
df = df.drop(['device', 'geoNetwork', 'totals', 'trafficSource', 'customDimensions', 'hits_x'], axis=1)

### 4. Cleaning existing columns

In [10]:
#Set dtype
df['pageviews'] = df['pageviews'].astype(int)
df['newVisits'] = df['newVisits'].astype(int)
df['bounces'] = df['bounces'].astype(int)
df['sessionQualityDim'] = df['sessionQualityDim'].astype(int)
df['timeOnSite'] = df['timeOnSite'].astype(int)
df['transactions'] = df['transactions'].astype(int)
df['transactionRevenue'] = df['transactionRevenue'].astype(int)
df['fullVisitorId'] = df['fullVisitorId'].astype(str)

In [11]:
#Create Day, month, year columns
df['date'] = pd.to_datetime(df['visitStartTime'], unit='s')
df['weekday'] = df['date'].dt.weekday
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df = df.drop(['date'], axis = 1)

In [12]:
#Make columns ints
df['isMobile'] = df['isMobile'].replace(True, 1)
df['isMobile'] = df['isMobile'].replace(False, 0)
df['isTrueDirect'] = df['isTrueDirect'].replace(False, 0)

In [13]:
#Not in this dataset or all the same value
df = df.drop(['socialEngagementType', 'visitId', 'visitStartTime', 'browserVersion', 'browserSize',
                 'operatingSystemVersion', 'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector',
                 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors',
                 'screenResolution', 'cityId', 'latitude', 'longitude', 'networkLocation', 
                 'adwordsClickInfo.criteriaParameters', 'adwordsClickInfo.isVideoAd', 'campaign', 'visits',
                 'bounces'], axis=1)

In [14]:
# Replacing Values
df.replace([np.nan, '(not set)', 'not available in demo dataset', '(not provided)'], 'Unknown', inplace = True)
df['keyword'].replace(['0'], 'Unknown', inplace = True)
df['isTrueDirect'].replace(['True'], 1, inplace = True)
df['adwordsClickInfo.slot'].replace(['0'], 'None', inplace = True)
df['adwordsClickInfo.adNetworkType'].replace(['0'], 'None', inplace = True)

### 5. Creating new columns

In [15]:
# Grouping fullVisitorId to find total transactions
df_trans = df.groupby(['fullVisitorId']).agg({'transactions': 'sum'}).reset_index()
df_trans.columns = ['fullVisitorId', 'total_transactions']
df = df.merge(df_trans, how='left', on=['fullVisitorId'])

In [16]:
# Creating a previousCustomer Column
df['previousCustomer'] = np.nan
df['previousCustomer'] = np.where(df['total_transactions'] == 0, 0, df['previousCustomer'])
df['previousCustomer'] = np.where(df['total_transactions'] == 1, 0, df['previousCustomer'])
df['previousCustomer'] = np.where(df['total_transactions'] > 1, 1, df['previousCustomer'])

In [17]:
# Creating the transaction_Complete Column that will be the column predicted
df['transaction_Complete'] = np.nan
df['transaction_Complete'] = np.where(df['transactionRevenue'] == 0, False, df['transactionRevenue'])
df['transaction_Complete'] = np.where(df['transactionRevenue'] > 1, True, df['transactionRevenue'])

### 6. Saving Clean Data

In [18]:
df.to_csv('df_clean.csv', index=False)